# Intelligent Data Analyst Assistant: Automated Chart Generation and Insight Explanation with RAG & GPT

Muh Amri Sidiq

## Introduction

In this project, I built an intelligent data analysis assistant that combines human-like reasoning with data-driven accuracy. By integrating Retrieval-Augmented Generation (RAG), GPT-4o, and Python data visualization libraries (like Matplotlib and Seaborn), the assistant is capable of understanding user questions, analyzing uploaded datasets (CSV or PDF), generating appropriate visualizations, and delivering meaningful insights all in real time.

Users can upload structured data such as sales reports or other tabular formats, and the system will automatically:
 - Detect relevant columns and schema,
 - Generate one or more visualizations (bar, pie, line, timeseries, scatter, histogram, correlation matrix),
 - Apply filters and aggregation as needed,
 - Interpret chart results using custom auto-analysis logic (generate_auto_answer()),
 - And finally present a concise, human-readable summary.

If the source data is from PDFs, the assistant retrieves and understands natural language passages and generates textual answers using GPT-based language reasoning.

This assistant mimics how a business analyst would explore data, extract patterns, and communicate findings making it suitable for both business users and data professionals. The result is a powerful tool that enhances productivity, reduces manual work, and showcases the power of combining LLMs with structured data pipelines.

## Workflow RAG

there are several steps to determine the RAG flow

### 1. Read Data

the data we use is tabular data that contains ecomerce transaction columns.

In [1]:
import pandas as pd

# Read the e-commerce dataset
# The dataset is assumed to be in the same directory as this script.
ecommerce = pd.read_csv("data/US  E-commerce records 2020.csv", encoding='cp1252')

### 2. Data to Text Transformation

So that the data can be used, which was originally tabular, it must first be converted into text form, so that LLM can read it. By combining all the columns of each row, the first step is to make the data of each row into 1 full sentence as a condition for LLM to read it.

In [2]:
# membuat sebuah fungsi
def merge_column(df, column_data):
    df['teks'] = df[column_data].astype('str').agg(' | '.join, axis = 1)
    return df

In [6]:
cols = ['Order Date', 'Row ID', 'Order ID', 'Ship Mode', 'Customer ID',
        'Segment', 'Country', 'City', 'State', 'Postal Code',
        'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
        'Sales', 'Quantity', 'Discount', 'Profit']

merge_column(df=ecommerce, column_data=cols)
ecommerce['teks'].head(5)

0    01-01-20 | 849 | CA-2017-107503 | Standard Cla...
1    01-01-20 | 4010 | CA-2017-144463 | Standard Cl...
2    01-01-20 | 6683 | CA-2017-154466 | First Class...
3    01-01-20 | 8070 | CA-2017-151750 | Standard Cl...
4    01-01-20 | 8071 | CA-2017-151750 | Standard Cl...
Name: teks, dtype: object

### 3. Search for Text Similarities

#### 3.1 Embedding Teks

At this stage we convert the text into a vector representation capturing the meaning of the text. In this case looking for similarities between one data and another.

Information about the similarity between one data and another is important because the initial goal of RAG is to retrieve data information that is relevant to the given question, in order to generate relevant LLM answers.

In [7]:
# prepare the embedding model to be used
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

`query =  "what segment makes the most purchases in the state of California?"`

`embedding_query_new = model.encode(query, convert_to_numpy=True)`

Setting up a query and embedding it for the information retrieval process.
The purpose of this code is to convert the text question ("query"). into a vector representation (embedding) so that it can be compared with data already indexed by FAISS. With this representation, the system can search for the most relevant rows of data based on similarity of meaning.

In [8]:
# prepare query & embedding new
query =  "what segment makes the most purchases in the state of California?"
embedding_query_new = model.encode(query, convert_to_numpy=True)

`embedding_dataframe = model.encode(ecommerce['teks'], convert_to_numpy=True)`

This code serves to convert the text in the text column of the ecommerce DataFrame into a vector representation (embedding) using the SentenceTransformer model. This embedding is important for performing semantic search with FAISS
 - model.encode(...): Calls the model to convert the text to embedding.
 - ecommerce[‘text’]: A column containing combined information from multiple
 - columns, prepared beforehand with a function like transform_data.

convert_to_numpy=True: Sets the output to be in numpy array format, which is required for use in FAISS index or other mathematical operations. The main purpose of this step is to prepare the data so that it can be searched based on similarity of meaning, not just keyword matching.

In [9]:
# prepare embedding for our data
embedding_dataframe = model.encode(ecommerce['teks'], convert_to_numpy=True)

`cosine_scores = util.cos_sim(embedding_query_new, embedding_dataframe)`

This code is used to calculate the semantic similarity between the query and all data in the embedding form.

Details:
 - embedding_query_new: The representation vector of the query that has been converted to embedding.
 - embedding_dataframe: Set of embeddings of all text data in the text column of the DataFrame.
 - util.cos_sim(...): A function from sentence_transformers.util that calculates cosine similarity, a measure of how similar two direction vectors are in multidimensional space.

Main objective: Find out how relevant each line of data is to the user's question based on closeness of meaning, not just word similarity.

Output:
 - cosine_scores produces a matrix with values between -1 and 1.
 - A value close to 1 means very similar, close to 0 means not similar, and a negative value means very different in semantic direction.

In [10]:
# calculate cosine similarity
cosine_scores = util.cos_sim(embedding_query_new, embedding_dataframe)
cosine_scores

tensor([[0.1494, 0.2476, 0.0978,  ..., 0.3054, 0.3561, 0.2677]])

#### 3.2: FAISS Indexing From Question

`import faiss, import numpy as np` This line imports two important libraries:
 - `faiss`: Used for indexing and similarity-based vector search.
 - `numpy`: Used for numeric array manipulation, such as vector normalization.

`embedding_dataframe = embedding_dataframe / np.linalg.norm(embedding_dataframe, axis=1, keepdims=True)`
`embedding_dataframe = embedding_dataframe.astype('float32')`
Manual normalization of embedding
 - Normalization is done so that each embedding vector has a length (norm) = 1.
 - This is important because FAISS uses inner product (dot product) to calculate similarity, and when vectors are normalized, dot product = cosine similarity
 - astype(‘float32’) is required because FAISS only accepts float32 data type.
   
   🔍 This normalization ensures the similarity search in FAISS is equivalent to cosine similarity.

`dimension = embedding_dataframe.shape[1]` Specifying Embedding Dimensions
 - Retrieves the number of dimensions of the embedding (for example, 384 if using the 'paraphrase-MiniLM-L6-v2' model).
 - This value is needed when creating the FAISS index:
    This dimension is the size of the vector space where all embeddings are placed.

In [11]:
import faiss
import numpy as np

# Calculate the cosine similarity scores with manually
embedding_dataframe = embedding_dataframe / np.linalg.norm(embedding_dataframe, axis=1, keepdims=True)
embedding_dataframe = embedding_dataframe.astype('float32')

# take value embedding
dimension = embedding_dataframe.shape[1]
dimension

384

`index = faiss.IndexFlatL2(dimension)` Creating and Populating the FAISS Index

This line makes the FAISS index use L2 (Euclidean) distance as the similarity metric.
Explanation:
 - IndexFlatL2 is a FAISS index type that calculates the Euclidean distance (L2 distance) between vectors.
 - dimension is the number of dimensions of each embedding vector (e.g. 384).
 - This index does not use fast search structures (such as IVF or HNSW) - suitable for small to medium datasets.

   📌 Use IndexFlatIP if you want to calculate cosine similarity with normalized vectors.

`index.add(embedding_dataframe)` This line adds all embedding vectors to the FAISS index.
Explanation:
 - embedding_dataframe contains all the encoded vectors of the text in the dataset.
 - Once added, the index is ready to be used for similarity-based queries.

   FAISS can now be used to find the most similar data to the given query.

In [12]:
index = faiss.IndexFlatL2(dimension)
index.add(embedding_dataframe)

The dimension results if compared back to the example query question above, will be as below.

In [13]:
query

'what segment makes the most purchases in the state of California?'

`embedding_index_query = model.encode([query])`

Explanation:
 - This code is used to convert a user's query into an embedding vector, so that it can be compared with vectors of previously indexed data.
 - `model.encode(...)`: A function of the SentenceTransformer that converts text into a numerical representation (embedding).
 - `[query]`: Given in list form because the model expects the input to be a list of strings, even if it is only one query.
 - The result is a 2-dimensional array (shape: [1, dimension]), which is the format FAISS needs for the search process.
 
 Main objective: Convert the user's question into a vector format, so that it can be compared against the entire data to find the most relevant answer.

In [14]:
embedding_index_query = model.encode([query])

`D, I = index.search(embedding_index_query, k = 2)`

Explanation:
 - This code is used to find the k closest (most similar) vectors in the FAISS index to the query that has been converted to embedding.
 - `embedding_index_query`: The encoding result vector of the user query.
 - `k = 2`: Retrieve the 2 closest results (top-2 most similar).
 - `index.search(...)`: FAISS function to perform a distance-based search (e.g. Euclidean if using IndexFlatL2, or cosine if using IndexFlatIP + normalization).

Output:
 - `D`: The distance matrix or similarity score between the query and the k closest results.
 - `I`: The row index matrix of the original data that is most similar to the query.

Main objective: Find the rows of data in the dataset that are most relevant to the user's query based on embedding.

In [15]:
D, I = index.search(embedding_index_query, k = 2)

`D:` the distance between the `query_vector` and the nearest vectors.

In [16]:
D

array([[44.061226, 44.409435]], dtype=float32)

`I:` index of most similar vectors

In [17]:
I

array([[3178,  411]], dtype=int64)

Function: Build FAISS Index with Cosine Similarity

Function Explanation:
The build_faiss_index_cosine(text) function is used to create a FAISS index based on cosine similarity, which is useful in the process of searching relevant data based on the similarity of text meaning.

Steps:
1. Embedding Text: `embedding = model.encode(teks , convert_to_numpy=True)`
   Convert a text list (text) into a numeric vector (embedding) using SentenceTransformer.
2. Normalization for Cosine Similarity: `embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)`,
`embedding = embedding.astype('float32')`. 
   - Normalize each vector to unit-norm so that dot product = cosine similarity.  - Convert to float32 as FAISS only supports this data type.
3. Create and Fill Index:
   `index = faiss.IndexFlatL2(dimension)`
    `index.add(embedding)`
   - Creating FAISS index is based on L2 distance, but since the vectors are already normalized, the L2 search is equivalent to cosine similarity.
   - dimension is the number of dimensions of the embedding vector (e.g. 384 or 768 depending on the model).
4. Return:
   - Returns the FAISS index and its embedding for use in the search process.

Notes:
- FAISS does not provide IndexFlatCosine function, so to do cosine similarity, we need to do manual normalization and still use IndexFlatL2.
- Make sure the input text is in the form of a list of strings so that the encode runs smoothly.

The main purpose of this function: Build a meaning-based fast search structure for use in RAG (Retrieval-Augmented Generation) or data-driven question and answer systems.

In [18]:
# make function to build FAISS index with cosine similarity
def build_faiss_index_cosine(teks):
    # Section for performing embeddings
    embedding = model.encode(teks , convert_to_numpy=True)

    # Perform cosine calculation
    embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
    embedding = embedding.astype('float32')

    # Indexing
    dimension = embedding.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embedding)

    return index, embedding

Function: Retrieve Relevant Data from FAISS Index

Function Explanation:
The retrieve() function is used to retrieve the most relevant rows of data against a query based on meaning similarity using the FAISS index.

Detailed Steps:
1. Encode and Normalize Query:

   `query_embedding = model.encode([query], convert_to_numpy=True)`
   `query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)`
   `query_embedding = query_embedding.astype("float32")`

   - The query is converted into a vector (embedding).
   - The query vector is normalized so that it can be compared with the embedding data based on cosine similarity.
   - Conversion to float32 as FAISS only supports this format.
2. Search the FAISS Index:
   
   `scores, indices = index.search(query_embedding, top_k)`

   - Searches the index to find the top_k most similar results.
   - scores: Similarity or distance values (higher if cosine similarity).
   - indices: Index of rows from the original data that are most relevant.
3. Fetch Data from DataFrame:
   
   `result_df = df.iloc[indices[0]].copy()`
   `result_df['similarity_score'] = scores[0]`

   - Retrieve rows from the original DataFrame (df) based on the FAISS result.
   - index.- Adds a new column similarity_score to display the similarity score.
4. Return Results:
   
   `return result_df`

   - Returns a DataFrame containing relevant data rows and similarity scores.

Final Destination:
This function is the core part of the RAG (Retrieval-Augmented Generation) system, which enables the model to answer the most relevant data-driven questions quickly and accurately.


In [19]:
# make function to retrieve data from FAISS index
def retrieve(query, index, df, top_k=3):
    # 1. Encode dan normalization query
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
    query_embedding = query_embedding.astype("float32")

    # 2. Search to FAISS
    scores, indices = index.search(query_embedding, top_k)

    # 3. Retrieve the data row according to the indexing result
    result_df = df.iloc[indices[0]].copy()
    result_df['similarity_score'] = scores[0]

    return result_df

Function: Generate Answers Using OpenAI GPT

Function Explanation:
The generate_answer() function is used to generate text-based answers using the OpenAI ChatCompletion API (GPT-4.1-mini model). It combines the user's question and relevant context data to generate an informative and specific answer.

 Line-by-Line Explanation:
1. Set the OpenAI API Key:
   `openai.api_key = api_key`

   The API key is set so that the request can be authenticated and the OpenAI service can be used.
2. Set up a Prompt System:

   `system_message = "Kamu adalah asisten cerdas yang menjawab pertanyaan berdasarkan data yang diberikan."`
   
   System messages to direct the model to act as a data-driven intelligent assistant.
3. Compose Prompts from Users:
   
   `user_message = f"""`
   `Pertanyaan: {query}`

   `Data yang relevan:`

   `{context}`
   
   `"""`

   Combining questions and relevant data (results from FAISS retrieval) into a prompt format for the model.

4. Send Request to OpenAI:
   
   `response = openai.ChatCompletion.create(`

    `model="gpt-4.1-mini",`
    
    `messages=[...],`
    
    `temperature=0.3,`
    
    `max_tokens=1000`
    
    `)`

    - Model: The model used (in this case GPT-4.1-mini).
    - messages: A list of messages to set the context of the conversation. 
    - temperature=0.3: Creativity control. The lower it is, the more deterministic the output.
    - max_tokens=1000: Limit on the length of the generated answer.

5. Returning Answers:
   
   `return response.choices[0].message["content"]`

   Retrieve the answer from the result provided by OpenAI and return it as text.

Function Objective:
Connecting the data retrieval process with GPT's reasoning capabilities, so that the system can answer questions based on semantically discovered information from the dataset.




In [20]:
import openai

def generate_answer(query, context, api_key):
    # to enter the api key of the generative model being used
    openai.api_key = api_key
    # to tell you specifically what the generative model needs to do.
    system_message = "Kamu adalah asisten cerdas yang menjawab pertanyaan berdasarkan data yang diberikan."
    # for users to input questions or data that they want to learn.
    user_message = f"""
    Pertanyaan: {query}

    Data yang relevan:
    {context}
    """
    response = openai.ChatCompletion.create(
        model="gpt-4.1-mini", # the model being used
        # system messages or to process the input data or user
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        # to adjust the level of randomness in the next word prediction
        temperature=0.3,
        # to set the maximum number of tokens that can be processed
        max_tokens=1000
    )
    return response.choices[0].message["content"]

Generate Answers Using GPT Based on Relevant Data

Explanation:
The generate_answer() function is called to answer a user-specific question based on the data set provided in the text fields of the ecommerce DataFrame.
Parameters:
 - `query`:
   The user question that the model wants to answer.
   In this example:
   “what segment makes the most purchases in the state of California?”

- `context`:
   A set of relevant data in text form (usually the result of multiple columns combined), used as context for the GPT model to answer based on real information.
   Here: ecommerce[‘text’].

- `api_key`:
  The API key used to access the OpenAI ChatCompletion API. Must be provided for the request to run.

In [ ]:
generate_answer(query= "What is the total sales for State = CA, Ship Mode = First Class, and Segment = Home Office?",
                context=  ecommerce['teks'],  
                api_key= "")

## Auto Detect Column

The `auto_detect_columns_info(df)` function aims to generate a summary of the data structure (schema) of a DataFrame df in text form, so that it can be used as a context for LLM models (such as GPT) to understand the data content.

In [21]:
def auto_detect_columns_info(df): 
    context_info = "Schema kolom dari data:\n" # Initialize the context_info string that will contain all the column information.
    for col in df.columns: # Loop to each column in the DataFrame
        try:
            sample_val = df[col].dropna().iloc[0] #  Take one instance of a value from a column that is not NaN (dropna()), to test if it is a date.
            is_date = False
            # Coba parse datetime
            try:
                pd.to_datetime(sample_val) # Try converting the instance value to datetime. If successful, it is considered that the column is of type time (datetime).
                is_date = True
            except:
                pass

            dtype = "datetime" if is_date else str(df[col].dtype)
        except:
            dtype = str(df[col].dtype) # If error when retrieving sample, fallback to default data type (df[col].dtype).

        # Ambil 3 sample unik (stringify agar tidak error dengan tipe aneh)
        sample = list(map(str, df[col].dropna().unique()[:3])) # Take the first 3 unique values from the column (without NaN), and convert them to strings for safe display (e.g. number, date, object).
        context_info += f"- {col} (type: {dtype}, sample: {sample})\n" # Add the detection result for this column to the context_info string.
    return context_info # Returns a string containing all the column information, which can be used as a context prompt for LLM.

## Generate Visualization

The execute_plot(args, df) function is an important component of Streamlit-based data visualization applications. Its purpose is to generate graphs according to the dynamic instructions of the LLM model or the user, based on the data df and the plotting parameters args.

What Does This Function Do?
This function:
 - Reads visualization parameters (args) such as plot type (plot_type), x/y column, aggregation (agg), and filter.
 - Prepares data (filter, change datetime type, aggregation).
 - Create a graph according to the plot type: bar, line, scatter, pie, timeseries, histogram, or correlation_matrix.
 - Display chart results in the UI with Streamlit's st.pyplot().

In [ ]:
def execute_plot(args, df):
    import matplotlib.pyplot as plt
    import seaborn as sns

    plot_type = args.get("plot_type") # Retrieve Parameters from args
    x = args.get("x")
    y = args.get("y")
    agg = args.get("agg", "sum")
    filters = args.get("filters", {})

    if not plot_type or plot_type not in ["bar", "line", "scatter", "pie", "timeseries", "histogram", "correlation_matrix"]: # Check if Plot Type is Valid
        st.warning(f"📛 Type plot '{plot_type}' not recognized or not yet supported.")
        return

    df_filtered = df.copy()

    # --- Convert date if there is a date field
    for col in df_filtered.columns:
        if "date" in col.lower() or "tanggal" in col.lower():
            try:
                df_filtered[col] = pd.to_datetime(df_filtered[col], errors='coerce')
            except:
                pass

    # --- Apply filters
    for col, val in filters.items():
        if col not in df_filtered.columns:
            continue
        if isinstance(val, list):
            df_filtered = df_filtered[df_filtered[col].isin(val)]
        elif isinstance(val, str) and val.isdigit() and 'date' in col.lower():
            df_filtered = df_filtered[df_filtered[col].dt.year == int(val)]
        else:
            df_filtered = df_filtered[df_filtered[col] == val]

    # --- Handle Empty Data
    if df_filtered.empty:
        st.warning("📭 No data matching the filter.")
        return

    # --- Aggregation (if required)
    if plot_type in ["bar", "line", "pie", "timeseries"] and x and y:
        if agg == "sum":
            grouped = df_filtered.groupby(x)[y].sum().reset_index()
        elif agg == "mean":
            grouped = df_filtered.groupby(x)[y].mean().reset_index()
        elif agg == "median":
            grouped = df_filtered.groupby(x)[y].median().reset_index()
        elif agg == "count":
            grouped = df_filtered.groupby(x)[y].count().reset_index()
        else:
            raise ValueError(f"Unsupported aggregation: {agg}")
    else:
        grouped = df_filtered

    # --- Plot
    plt.figure(figsize=(10, 5))

    if plot_type == "bar":
        plt.bar(grouped[x], grouped[y])
        plt.title(f"Bar Chart of {y} by {x} ({agg})")
        plt.xticks(rotation=45)

    elif plot_type == "line":
        plt.plot(grouped[x], grouped[y], marker='o')
        plt.title(f"Line Chart of {y} by {x} ({agg})")

    elif plot_type == "scatter":
        plt.scatter(grouped[x], grouped[y])
        plt.title(f"Scatter Plot: {y} vs {x}")

    elif plot_type == "pie":
        plt.pie(grouped[y], labels=grouped[x], autopct='%1.1f%%')
        plt.title(f"Pie Chart of {y} by {x} ({agg})")

    elif plot_type == "timeseries":
        grouped[x] = pd.to_datetime(grouped[x], errors='coerce')
        grouped = grouped.sort_values(x)
        plt.plot(grouped[x], grouped[y])
        plt.title(f"Time Series: {y} over {x} ({agg})")

    elif plot_type == "histogram":
       numeric_cols = df_filtered.select_dtypes(include='number').columns.tolist()

       if y:
           if isinstance(y, list):  # can be from LLM as a list
               cols_to_plot = [col for col in y if col in numeric_cols]
           else:
               cols_to_plot = [y] if y in numeric_cols else []
       else:
           cols_to_plot = numeric_cols

       if not cols_to_plot:
           st.error("📛 No numeric column can be used for histogram.")
           return

       for col in cols_to_plot:
           sns.histplot(df_filtered[col], kde=True)
           plt.title(f"Histogram of {col}")
           plt.xlabel(col)
           plt.tight_layout()
           st.pyplot(plt)
           plt.clf()
       return


    elif plot_type == "correlation_matrix":
        numeric_df = df_filtered.select_dtypes(include='number')
        corr = numeric_df.corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm')
        plt.title("Correlation Matrix")

    else:
        st.error(f"Type plot '{plot_type}' not recognized or not yet supported.")
        return

    plt.tight_layout()
    st.pyplot(plt)

Benefits of This Function:
 - Flexible to various data types and queries.
 - Supports automatic plotting of LLM instructions.
 - Can be combined with analysis such as generate_auto_answer() to create narrative answers from graphs.

## Interpret Chart

The function `generate_auto_answer(df, args)` is a data-driven automatic analysis that:
- Reads the visualization type (plot_type) and parameters from args (e.g. from LLM).
- Performs basic numerical/statistical analysis depending on the graph type.
- Generates narrative (natural language) answers that match the content of the graph that will be/was just displayed.
- Very useful for your system to explain graphs without LLM, based only on the data and visualization parameters.

In [22]:
def generate_auto_answer(df, args):
    try:
        plot_type = args.get("plot_type")
        x = args.get("x")
        y = args.get("y")
        agg = args.get("agg", "sum")
        filters = args.get("filters", {})

        # Filter data
        for k, v in filters.items():
            if k in df.columns:
                df = df[df[k] == v]

        if df.empty:
            return "Empty data after filtering."

        # Bar, Line, Timeseries
        if plot_type in ["bar", "line", "timeseries"]:
            if x not in df.columns or y not in df.columns:
                return f"columns {x} or {y} Not found in the data."

            grouped = df.groupby(x)[y].agg(agg).reset_index()
            grouped = grouped.sort_values(by=y, ascending=False)
            top = grouped.iloc[0]
            percent = round(100 * top[y] / grouped[y].sum(), 2)

            return f"Category '{top[x]}' have {agg} {y.lower()} highest, i.e {top[y]:,.2f} ({percent}%) compared to other categories."

        # Scatter plot
        elif plot_type == "scatter":
            if x not in df.columns or y not in df.columns:
                return f"columns {x} or {y} Not found in the data."

            corr = df[[x, y]].corr().iloc[0, 1]
            return f"There is a correlation of {corr:.2f} between '{x}' and '{y}'."

        # Pie chart
        elif plot_type == "pie":
            if x not in df.columns or y not in df.columns:
                return f"columns {x} or {y} Not found in the data."

            pie_group = df.groupby(x)[y].agg(agg).reset_index()
            top = pie_group.sort_values(by=y, ascending=False).iloc[0]
            percent = round(100 * top[y] / pie_group[y].sum(), 2)
            return f"Category '{top[x]}' has contribution {agg} {y.lower()} highest i.e {top[y]:,.2f} ({percent}%)."

        # Correlation Matrix
        elif plot_type == "correlation_matrix":
            numeric_cols = df.select_dtypes(include='number').columns
            corr_matrix = df[numeric_cols].corr()
            high_corr = corr_matrix.where(~np.eye(len(corr_matrix),dtype=bool)).abs().stack().sort_values(ascending=False)
            if not high_corr.empty:
                top_pair = high_corr.idxmax()
                value = high_corr.max()
                return f"The highest correlation is between '{top_pair[0]}' and '{top_pair[1]}', with value {value:.2f}."
            else:
                return "No significant correlation found between numeric columns."

        # Histogram
        elif plot_type == "histogram":
            numeric_cols = df.select_dtypes(include='number').columns.tolist()
            return f"The histogram will show the distribution of the following numerical columns: {', '.join(numeric_cols)}."

        else:
            return f"Type plot '{plot_type}' not recognized."

    except Exception as e:
        return f"Fails to generate auto-answer: {e}"

When is this function used?
 - When you want to explain the graph automatically (without LLM).
 - To synchronize plots and answers, especially when datasets are uploaded freely (they can be different).
 - As a fallback if there is no good answer from LLM, or as a complement to the answer from LLM.

## Conclusion

This project demonstrates a fully automated Data Analytics Assistant that integrates:
- Retrieval-Augmented Generation (RAG) for understanding both structured (CSV) and unstructured (PDF) data.
- Multi-chart generation with support for various plot types (bar, line, pie, scatter, time series, histogram, and correlation matrix).
- Dynamic function calls generated via LLM (GPT-4o) based on user questions and column schema.
- A robust fallback mechanism using a custom generate_auto_answer() function to provide reliable and explainable data insights, even without depending entirely on LLM-generated text.

Key strengths of the system include:
- Flexibility to handle any uploaded dataset regardless of column names or types.
- Smart syncing of visualizations with interpretable natural language answers
- Compatibility with both quantitative data (CSV) and qualitative text (PDF) in a unified Streamlit interface.

This solution reflects not only strong integration of LLM + Data Science + UI, but also real-world considerations such as:
- Handling missing columns and schema mismatches.
- Producing meaningful insights for business questions.
- Ensuring plot and answer coherence across multiple chart types.

✅ In summary, this portfolio showcases how LLMs can be enhanced with structured logic to build intelligent, general-purpose data assistants—ready for business analytics, automated reporting, or even client-facing dashboards.